# Problem description

Hi, I'm SalvadOR, responsible for creating a school timetable to organize classes, teachers, and rooms for an upcoming semester.

We have some strict requirements to meet. This is a very hard problem we face every year, and I need your help to design an optimal timetable.

Here's the situation:

* There are 4 classes, each requiring specific teaching sessions.
* There are 4 teachers, and each teacher has assigned subjects to teach.
* There are 4 rooms, and only one class can occupy a room during any given period.
* The timetable spans 30 periods, and we must ensure that all requirements are met without any conflicts or overlaps.

I desperately need a timetable that satisfies all requirements (each class meets with the right teacher in the right room the required number of times), avoiding any type of clashes such as double-booking a teacher, room, or class during the same period.

We say a timetable is optimized when it minimizes idle periods and maximizes resource utilization (teachers and rooms).

Can you help me solve this problem?

# Problem data

Here are the first 14 lines of the provided data file:

    # A matrix specifying how often each class-teacher combination needs to meet in specific rooms.

    # For instance:
    # A "6" in the third row of the matrix means Class 3 must meet Teacher 4 in Room 1 six times.
    # A "5" in the fifth row means Class 1 must meet Teacher 2 in Room 2 five times.

    # The matrix is of the following format:
    # teacher  1 2 3 4
    # --------------------
    # class 1: 2 2 1 2
    #       2: 1 1 1 2
    #       3: 1 1 1 6
    #       4: 2 2 3 2 room 1
    # ------------

# Solution

We build a graph whose vertices are quadruples (room, class, teacher, lesson id), with an edge if at least one of the first three coincide (we produce unique lesson ids). Then the problem reduces to finding a graph coloring, the colors representing different periods.

It turns out that NetworkX's greedy coloring does not produce a time table that fits in 30 periods. Either a specialized algorithm or another approach is needed.

In [1]:
from collections.abc import Iterable

import networkx as nx


CLASSES = ROOMS = TEACHERS = 4


def iter_data_file_numbers(path: str) -> Iterable[int]:
    with open(path) as file:
        for row in file:
            if row.startswith('#'):
                continue

            for num_str in row.split():
                yield int(num_str)


def read_data(path: str) -> nx.Graph:
    it = iter(iter_data_file_numbers(path))
    graph = nx.Graph()
    lesson_id = 0

    for r in range(1, ROOMS + 1):
        for i in range(1, CLASSES + 1):
            for j in range(1, TEACHERS + 1):
                for _ in range(next(it)):
                    lesson_id += 1
                    graph.add_node(lesson_id, room=r, class_=i, teacher=j)

    for u in graph.nodes:
        for v in graph.nodes:
            if u != v and len(graph.nodes[u].items() & graph.nodes[v].items()) > 0:
                graph.add_edge(u, v)

    return graph


graph = read_data('data/04.txt')
coloring = nx.coloring.greedy_color(graph)
n_periods = len(set(coloring.values()))
n_periods

39